In [1]:
import cv2, ast
import torchvision
import matplotlib.pyplot as plt
from os.path import isfile
import torch.nn.init as init
import torch
import torch.nn as nn
import numpy as np
import pandas as pd 
import os
from PIL import Image, ImageFilter
from sklearn.model_selection import train_test_split, StratifiedKFold
from torch.utils.data import Dataset
from torchvision import transforms
from torch.optim import Adam, SGD, RMSprop
import time
from torch.autograd import Variable
import torch.functional as F
from tqdm import tqdm
from sklearn import metrics
import urllib
import pickle
#import cv2
import torch.nn.functional as F
from torchvision import models
import seaborn as sns
import random
#from apex import amp
import sys

In [2]:
from skimage import exposure

In [3]:
import segmentation_models_pytorch as smp

In [4]:
val_df = pd.read_pickle('lung_cancer_test.pkl')
val_df.head()

,label1,mask,hu_array,hu_array_old
0,LR2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...","[[-1024.0, -1024.0, -1024.0, -1024.0, -1024.0,..."
1,LR2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...","[[-1024.0, -1024.0, -1024.0, -1024.0, -1024.0,..."
2,LR2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...","[[-1024.0, -1024.0, -1024.0, -1024.0, -1024.0,..."
3,LR2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...","[[-1024.0, -1024.0, -1024.0, -1024.0, -1024.0,..."
4,LR2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...","[[-1024.0, -1024.0, -1024.0, -1024.0, -1024.0,..."


In [5]:
model = smp.Unet(
    encoder_name="vgg11",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
).cuda()

In [6]:
model.eval()

Unet(
  (encoder): VGGEncoder(
    (features): Sequential(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (12): ReLU(inplace=True)
      (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (14): ReLU(inplace=True)
      (15): MaxPool2d(kernel

In [7]:
import numpy as np

def calculate_dice_coefficient(mask_true, mask_pred):
    intersection = np.sum(mask_true * mask_pred)
    union = np.sum(mask_true) + np.sum(mask_pred)
    
    dice_coefficient = (2.0 * intersection) / (union + 1e-8)  
    
    return dice_coefficient

def precision_score(groundtruth_mask, pred_mask):
    intersect = np.sum(pred_mask*groundtruth_mask)
    total_pixel_pred = np.sum(pred_mask)
    precision = np.mean((intersect++ 1e-8)/(total_pixel_pred++ 1e-8))
    return precision

def recall_score(groundtruth_mask, pred_mask):
    intersect = np.sum(pred_mask*groundtruth_mask)
    total_pixel_truth = np.sum(groundtruth_mask)
    recall = np.mean((intersect+ 1e-8)/(total_pixel_truth+ 1e-8))
    return recall

def calculate_iou(mask_true, mask_pred):
    intersection = np.sum(mask_true * mask_pred)
    union = np.sum(mask_true) + np.sum(mask_pred) - intersection
    
    iou = (intersection + 1e-8) / (union + 1e-8)  
    
    return iou
from scipy.spatial.distance import cdist

def hausdorff_distance(mask1, mask2):
    # Получение координат точек в масках
    coords_mask1 = np.transpose(np.nonzero(mask1))
    coords_mask2 = np.transpose(np.nonzero(mask2))

    # Вычисление расстояний между всеми точками в двух масках
    distances_mask1_to_mask2 = cdist(coords_mask1, coords_mask2)
    distances_mask2_to_mask1 = cdist(coords_mask2, coords_mask1)

    # Нахождение максимального расстояния для каждой маски
    max_distance_mask1 = np.max(np.min(distances_mask1_to_mask2, axis=1))
    max_distance_mask2 = np.max(np.min(distances_mask2_to_mask1, axis=1))

    # Вычисление дистанции Хаусдорфа
    hausdorff_distance = max(max_distance_mask1, max_distance_mask2)

    return hausdorff_distance

In [8]:
def model_evaluation(pretrained_weights, method_name):
    model.load_state_dict(torch.load(pretrained_weights))
    model.eval()
    print('augmentation method name: ' + method_name)
    d = []
    iou = []
    prec = []
    rec = []
    labels = val_df['label1'].unique()
    for j in labels:
        dices = []
        test = val_df.loc[val_df['label1'] == j].reset_index(drop=True)
        for i in range(len(test)):
            #new_shape = [224,224]
            mask = test['mask'][i]
            img = test['hu_array'][i]
            img = (img-np.min(img))/(np.max(img)-np.min(img))
            img = exposure.equalize_adapthist(img/np.max(img))
            img = img.astype(np.float64)
            img = transforms.ToTensor()(img)
            img = torch.unsqueeze(img, 0)
            img = img.float()
            img = img.cuda()
            masks = mask.astype(int)
            outputs = model(img)
            outputs = torch.sigmoid(outputs)
            try:
                mask_pred = (outputs.cpu().detach().numpy() >= 0.5) ** 2
            except:
                mask_pred = np.zeros([512,512])
            dices.append(calculate_dice_coefficient(masks, mask_pred))
            d.append(calculate_dice_coefficient(masks, mask_pred))
            iou.append(calculate_iou(masks, mask_pred))
            prec.append(precision_score(masks, mask_pred))
            rec.append(recall_score(masks, mask_pred))
        print("Dice for " + j + " " + str(np.mean(dices))+" " +str(len(dices)))
    print("Average") 
    print("DICE " + str(np.mean(d)))
    print("IoU " + str(np.mean(iou)))
    print("precision " + str(np.mean(prec)))
    print("recall "+str(np.mean(rec)))

In [11]:
model_evaluation('U-Net with VGG/Netdataaug_12.pth','U-Net')

augmentation method name: U-Net
Dice for LR2 0.1166048928253063 35
Dice for LR3 0.3952044555588205 37
Dice for LR4A 0.3395490924691667 97
Dice for LR4B 0.612629272338904 95
Average
DICE 0.41605976873586037
IoU 0.3566316357515479
precision 0.8112904057792768
recall 0.4441356389658143
